In [ ]:
! pip install torch torchvision torchaudio
! pip install transformers
! pip install pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 22.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
import pandas as pd

file_path = 'path_to_your_file.xlsx'
df_ = pd.read_excel(file_path)


df_['combined_text'] = df_['text2'].fillna("").astype('str')

df = df_.groupby('class', group_keys=False).apply(lambda x: x.sample(frac=0.3, random_state=42))

df = df.reset_index(drop=True)

print("Размер исходного датасета:", len(df_))
print("Размер выборки 30%:", len(df))
print("Примеры из выборки:")
print(df.head())

Размер исходного датасета: 226282
Размер выборки 30%: 67885
\Примеры из выборки:
                                               text1  \
0  \n\nФорма грудной клетки: обычная\nЛегочные по...   
1  \n\nФорма грудной клетки: обычная\nЛегочные по...   
2  \nВыполнена флюорография органов грудной клетк...   
3  \nКлиническая информация: в 2х проекциях\nФорм...   
4  \n Клиническая информация:\n Выполнена рентген...   

                                               text2  class  \
0             \nПатологических изменений не выявлено      0   
1  \nОчаговые и инфильтративные изменения не выяв...      0   
2            \nПатологических изменений не выявлено.      0   
3  \nРентгенологических признаков патологических ...      0   
4  \n Патологических изменений органов грудной кл...      0   

                                       combined_text  
0             \nПатологических изменений не выявлено  
1  \nОчаговые и инфильтративные изменения не выяв...  
2            \nПатологических изменени

<ipython-input-2-cbea0f77b4cc>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df_.groupby('class', group_keys=False).apply(lambda x: x.sample(frac=0.3, random_state=42))


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['combined_text'].tolist(),
    df['class'].tolist(),
    test_size=0.2,
    random_state=42
)



In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
import torch

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextDataset(train_encodings, train_labels)
test_dataset = TextDataset(test_encodings, test_labels)


In [ ]:
from transformers import BertForSequenceClassification
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

model = model.to(device)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer
import wandb
from transformers.integrations import WandbCallback
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted'
    )
    acc = accuracy_score(labels, predictions)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


wandb.init(mode='offline', project='my_project')


training_args = TrainingArguments(
    output_dir='./results',
    run_name='my_unique_run_name',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True,
    dataloader_num_workers=4,
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=1000,
    gradient_accumulation_steps=2,
    report_to="wandb",
    load_best_model_at_end=True,
    metric_for_best_model='f1'
)


if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)

trainer = Trainer(
    model=model.to(device),
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[WandbCallback()]
)

trainer.train()

final_metrics = trainer.evaluate(test_dataset)
print("Финальные метрики на тестовых данных:")
print(f"Accuracy: {final_metrics['eval_accuracy']:.4f}")
print(f"F1-score: {final_metrics['eval_f1']:.4f}")
print(f"Precision: {final_metrics['eval_precision']:.4f}")
print(f"Recall: {final_metrics['eval_recall']:.4f}")

trainer.save_model('./saved_model')
tokenizer.save_pretrained('./saved_model')

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
You are adding a <class 'transformers.integrations.integration_utils.WandbCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
WandbCallback


Using device: cuda


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.121400,0.062311,0.984975,0.984974,0.984989,0.984975
1000,0.157300,0.183639,0.952272,0.952225,0.953405,0.952272
1500,0.094800,0.230517,0.945496,0.945490,0.945541,0.945496
2000,0.113900,0.238211,0.941887,0.941885,0.941894,0.941887
2500,0.240700,0.221740,0.944539,0.944534,0.944568,0.944539
3000,0.256600,0.180850,0.952420,0.952391,0.954368,0.952420
3500,0.068100,0.109759,0.976946,0.976945,0.976960,0.976946
4000,0.108900,0.114291,0.972601,0.972585,0.973199,0.972601
4500,0.691500,0.682855,0.522280,0.374316,0.754047,0.522280
5000,0.063700,0.104093,0.978493,0.978488,0.978663,0.978493


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: T


Финальные метрики на тестовых данных:
Accuracy: 0.9859
F1-score: 0.9859
Precision: 0.9860
Recall: 0.9859


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json')

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


{'eval_loss': 0.07611311227083206,
 'eval_accuracy': 0.9858584370626795,
 'eval_f1': 0.9858556565027133,
 'eval_precision': 0.9860019558437415,
 'eval_recall': 0.9858584370626795,
 'eval_runtime': 68.4657,
 'eval_samples_per_second': 198.304,
 'eval_steps_per_second': 12.4,
 'epoch': 2.999263514508764}